In [1]:
import pickle

old_tag = "somoto_0.8_main" # used for pickle name
old_outputPath = "output/SBBGCA_Clustering/somoto_1022/main/"
pickleDir = old_outputPath + "pickle/"

residual_group_pickle = pickleDir + old_tag +"_residual.pickle"

thresholdValue = 0

new_tag = "somoto_0.8_main_residualTree"
new_outputPath = "output/SBBGCA_Clustering/somoto_1022/main/residual_tree_merge/"
new_pickleDir = new_outputPath + "pickle/"

import os
if not os.path.isdir(new_outputPath): os.makedirs(new_outputPath)
if not os.path.isdir(new_pickleDir): os.makedirs(new_pickleDir)

In [2]:
% run SBBGCA.ipynb

with open(residual_group_pickle, 'rb') as handle:
    residual = pickle.load(handle)

    residualReps_dict = dict()
    for key, value in residual.items():
        clusterName = value[0][0]
        motifsList = value[0][1]
        members = value[1]
        if(len(members) <= 1): continue # skip those single sample which never merged in RasMMA (t=0.8)
        
        featureHooklog = list()
        for motif in motifsList:
            featureHooklog.extend(motif[0]) # motif[0] is apiSequences
        
        R = (clusterName, [(featureHooklog, 0, len(featureHooklog)-1)] )
        clusterMembers = {clusterName}
        
        residualReps_dict[int(clusterName.split('G')[1])] = (R, clusterMembers)        
    
    if len(residualReps_dict.keys()) > 1: # at least two trees
        
        intermediatePool, initialDict, roundInfos, residualMessage = tryMergeResidualReps(residualReps_dict, new_tag, new_outputPath, thresholdValue)
        # saving intermediatePool as pickle file
        with open(new_pickleDir + new_tag + '_intermediate.pickle', 'wb') as handle:
            pickle.dump(intermediatePool, handle, protocol=pickle.HIGHEST_PROTOCOL)

        # saving initialNames dict as pickle file
        with open(new_pickleDir + new_tag + '_initialDict.pickle', 'wb') as handle:
            pickle.dump(initialDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        # saving round information dict as pickle file
        with open(new_pickleDir + new_tag + '_roundInfos.pickle', 'wb') as handle:
            pickle.dump(roundInfos, handle, protocol=pickle.HIGHEST_PROTOCOL)

        if(residualMessage is not None): # this shouldn't occur when thresholdValue is 0
            with open(new_pickleDir + new_tag + '_residual.pickle', 'wb') as handle:
                pickle.dump(residualMessage, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        print("no residual tree candidates.")

-- Start Clustering --
Threshold set = 0
-- Finish Clustering --


In [3]:
# reverse clusterID by hooklogName

def findClusterID(nameDict, hooklogName):
    for key, value in nameDict.items():
        if(value == hooklogName):
            return key
    return hooklogName


# convert memberSet to List type
def getMemberList(memberSet, nameDict):
    memberList = list()
    while(len(memberSet)>0):
        member = memberSet.pop()
        clusterID = findClusterID(nameDict, member)
        memberList.append(clusterID)
    return memberList

In [4]:
# read the results from pickle files
with open(new_pickleDir + new_tag + '_intermediate.pickle', 'rb') as handle:
    intermediate = pickle.load(handle)
with open(new_pickleDir + new_tag + '_initialDict.pickle', 'rb') as handle:
    initialDict = pickle.load(handle)
with open(new_pickleDir + new_tag + '_roundInfos.pickle', 'rb') as handle:
    roundInfos = pickle.load(handle)

initialNameDict = getInitialNameDict(initialDict)
print("Original Names : ", initialNameDict)
print("round informations", roundInfos)

intermediate_list = sorted(intermediate.items(), key=lambda x:x[0])
for item in intermediate_list:
    value = item[1] # get original dict value
    score = value[0]
    clusterName = value[1][0]
    memberSet = value[2] # members set
    memberList = getMemberList(memberSet, initialNameDict)
    print(clusterName, " : ", memberList)

Original Names :  {'G24': 'G24', 'G29': 'G29', 'G31': 'G31'}
round informations {0: ['G24', 'G29', 'G31'], 1: ['G3'], 2: ['G4']}
G3  :  ['G24', 'G29']
G4  :  ['G3', 'G31']
